# Find top 10000 vocab + Logistic Rgression


In [87]:
import pandas as pd
import joblib
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from scipy import sparse
import json
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

Task 1: Evaluation on simple logistic regression model, on fake news corpus test set


In [61]:
#Load the logistic regression model 
logistic_model = joblib.load('logistic_model.joblib')

c:\Users\thom0c49\AppData\Local\anaconda3\Lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.6.1 when using version 1.5.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [62]:
#Load data test set 
X_test_scaled = sparse.load_npz("X_test_scaled.npz")
y_test = pd.read_csv("y_test.csv")

In [83]:
# Evaluate with test
y_pred = logistic_model.predict(X_test_scaled)

f1 = f1_score(y_test, y_pred, average='weighted')
print("Total F1-score:", f1)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Total F1-score: 0.8093849082493505
Classification Report:
              precision    recall  f1-score   support

        fake       0.90      0.82      0.86     35942
    reliable       0.64      0.77      0.70     15085

    accuracy                           0.80     51027
   macro avg       0.77      0.80      0.78     51027
weighted avg       0.82      0.80      0.81     51027

Confusion Matrix:
[[29411  6531]
 [ 3435 11650]]


Task: 2, evaluate the logistic model on liar data 

Load "LIAR" dataset

In [64]:
# Load evaluation data from LIAR-dataset
liar_data = pd.read_csv("liar_full_dataset_preprocessed.csv", usecols=['type', 'content'], dtype=str)

In [65]:
type_mapping_liar = {
    'true': 'reliable', 
    'false': 'fake', 
    'half-true': 'fake', 
    'pants-fire': 'fake', 
    'barely-true': 'reliable',
    'mostly-true': 'reliable'
}

In [66]:
liar_data["label"] = liar_data["type"].map(type_mapping_liar)

In [67]:
# Load the top_10000_words
with open("top_10000_words.json", "r", encoding="utf-8") as fil:
    top_10000_words = json.load(fil)

In [68]:
vectorizer = CountVectorizer(vocabulary=top_10000_words)

In [69]:
# Transform liar data
X_liar = vectorizer.transform(liar_data['content'])

In [88]:
# Load scaleren
scaler = joblib.load('scaler.pkl')

In [89]:
# Scale data
X_liar_scaled = scaler.transform(X_liar)

In [96]:
y_liar = liar_data['label']
y_liar_df = y_liar.to_frame()


In [92]:
y_pred_liar = logistic_model.predict(X_liar_scaled)

In [97]:
#Evaluation on liar data 
f1_liar = f1_score(y_liar_df, y_pred_liar, average='weighted')
print("Total F1-score:", f1_liar)

print("Classification Report:")
print(classification_report(y_liar, y_pred_liar))

print("Confusion Matrix:")
print(confusion_matrix(y_liar, y_pred_liar))

TypeError: '<' not supported between instances of 'float' and 'str'

# Logistic Regression

In [25]:
from sklearn.preprocessing import MaxAbsScaler
import re
import numpy as np
import pandas as pd
import nltk
from collections import Counter
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

# ---- 3. Convert text to Bag-of-Words feature matrix ----

content_as_lists = data['content'].apply(ast.literal_eval)

all_words = content_as_lists.explode().tolist()

word_counts = Counter(all_words)

top_10000_words = [word for word, _ in word_counts.most_common(10000)]

vectorizer = CountVectorizer(vocabulary=top_10000_words)


In [ ]:
# Transform training, valid og test data
X_train = vectorizer.transform(train['content'])
X_valid = vectorizer.transform(valid['content'])
X_test = vectorizer.transform(test['content'])

# Transform LIAR test data to Bag-of-Words
# X_test_liar = vectorizer.transform(liar_test_data['content'])

In [ ]:
scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)
X_test_scaled = scaler.transform(X_test)
# LIAR not scaled

# Labels
y_train = train['label']
y_valid = valid['label']
y_test = test['label']

y_test_liar = liar_test_data['label']

# ---- 4. Train Logistic Regression model ----

clf = LogisticRegression(max_iter=100000, solver='saga', random_state=42)
clf.fit(X_train_scaled, y_train)

# ---- 5. Evaluate model ----

y_pred_liar = clf.predict(X_test_liar)

# ---- 6. Evaluate model on LIAR test. Notice that it isn't scaled since scaling gave an even worse result ----

f1 = f1_score(y_test_liar, y_pred_liar, average='weighted')  # eller 'macro'
print("Total F1-score: LIAR dataset:", f1)

print("Classification Report:")
print(classification_report(y_test_liar, y_pred_liar))

from sklearn.metrics import confusion_matrix

print("Confusion Matrix:")
print(confusion_matrix(y_test_liar, y_pred_liar))



# ---- Original test with test set (remove # for comparison)----
# print("Original test with test set - classification report")
# print(classification_report(y_test, clf.predict(X_test)))

Total F1-score: 0.49159389082685817
Classification Report:
              precision    recall  f1-score   support

        fake       0.48      0.32      0.38       606
    reliable       0.52      0.69      0.59       661

    accuracy                           0.51      1267
   macro avg       0.50      0.50      0.49      1267
weighted avg       0.50      0.51      0.49      1267

Confusion Matrix:
[[191 415]
 [207 454]]
Original test with test set - classification report
              precision    recall  f1-score   support

        fake       0.83      0.75      0.79       676
    reliable       0.78      0.85      0.81       694

    accuracy                           0.80      1370
   macro avg       0.80      0.80      0.80      1370
weighted avg       0.80      0.80      0.80      1370

